In [3]:
#All_samples_extracted_3UTR_coverage_dict=All_samples_Target_3UTR_coverages
# np.array(All_Samples_Total_depth),
# UTR_events_dict

In [4]:
import numpy as np
import os
import sys
import datetime

import scipy as sp
import scipy.stats

from bisect import bisect
from statsmodels.stats.multitest import multipletests

import math

In [5]:
def parse_cfgfile(cfg_file):
    '''Parse configure file
    '''
    Group1_Tophat_aligned_file=''
    Group2_Tophat_aligned_file=''
    output_directory=''
    Annotated_3UTR_file=''
    Output_result_file=''
    Num_least_in_group1_local=''
    Num_least_in_group2_local=''
    Coverage_cutoff_local = ''
    FDR_cutoff_local = ''
    Fold_change_cutoff_local = ''
    PDUI_cutoff_local = ''
    
    for line in open(cfg_file,'r'):
        if line[0] == '\n' or line[0] == '#':
            comments = line;
        else:
            line = line.rstrip();
            command = line.split('=');
            if command[0] == 'Group1_Tophat_aligned_Wig':
                Group1_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Group2_Tophat_aligned_Wig':
                Group2_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Output_directory':
                output_directory = command[1]
                if output_directory[-1] != '/':
                    output_directory += '/'
            if command[0] == 'Annotated_3UTR':
                Annotated_3UTR_file = command[1]
            if command[0] == 'Output_result_file':
                Output_result_file = command[1]
            
            ##Parameters
            if command[0] == 'Num_least_in_group1':
                Num_least_in_group1_local = command[1]
            if command[0] == 'Num_least_in_group2':
                Num_least_in_group2_local = command[1]
            if command[0] == 'Coverage_cutoff':
                Coverage_cutoff_local = command[1]
            if command[0] == 'FDR_cutoff':
                FDR_cutoff_local = command[1]
            if command[0] == 'Fold_change_cutoff':
                Fold_change_cutoff_local = command[1]
            if command[0] == 'PDUI_cutoff':
                PDUI_cutoff_local = command[1]
            
    
    if Group1_Tophat_aligned_file=='':
        print("No Tophat aligned BAM file for group 1!", file=sys.stderr)
        exit(1)
    if Group2_Tophat_aligned_file=='':
        print("No Tophat aligned BAM file for group 2!", file=sys.stderr)
        exit(1)
    if output_directory=='':
        print("No output directory!", file=sys.stderr)
        exit(1)
    if Annotated_3UTR_file=='':
        print("No annotated 3' UTR file!", file=sys.stderr)
        exit(1)
    if Output_result_file=='':
        print("No result file name!", file=sys.stderr)
        exit(1)
    return Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local


In [6]:
Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local = parse_cfgfile('/home/li/桌面/PROJECT6/apasite_predict2/DAPARS/DATA/DaPars_test_data_configure.txt')

In [7]:
Group1_Tophat_aligned_file=['../DATA/Condition_A_chrX.wig']
num_group_1 = len(Group1_Tophat_aligned_file)

In [8]:
Annotated_3UTR_file='../DATA/hg19_refseq_extracted_3UTR.bed'

In [9]:
# All_Sample_files = Group1_Tophat_aligned_file[:]
# All_Sample_files.extend(Group2_Tophat_aligned_file)
All_Sample_files=['../DATA/Condition_A_chrX.wig', '../DATA/Condition_B_chrX.wig']

In [10]:
for line in open(Annotated_3UTR_file,'r'):
    # print(line)
    #strip() 方法用于移除字符串头尾指定的字符
    fields = line.strip('\n').split('\t')
    print(fields)
    break

['chr14', '50792327', '50792946', 'NM_001003805|ATP5S|chr14|+', '0', '+']


In [11]:
curr_chr = fields[0]
region_start = int(float(fields[1]))
region_end   = int(float(fields[2]))
UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
UTR_pos

'chr14:50792327-50792946'

In [12]:
end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
end_shift

124

In [13]:
## 加载所有样品的覆盖率
## Load coverage for all samples
All_samples_extracted_3UTR_coverage_dict = {}
for curr_wig_file in All_Sample_files:
    break
curr_wig_file

'../DATA/Condition_A_chrX.wig'

In [14]:
for line in open(curr_wig_file,'r'):
    break
line

'chrX\t0\t193010\t0\n'

In [15]:
fields = line.strip('\n').split('\t')
fields

['chrX', '0', '193010', '0']

In [16]:
chrom_name = fields[0]
chrom_name

'chrX'

In [17]:
region_start = int(float(fields[1]))
region_end   = int(float(fields[2]))
region_end

193010

In [18]:
curr_strand  = fields[-1]